# 4교시 조인 연산

### 목차
* [1. 조인 유형](#1.-조인-유형)
* [2. Inner Join](#2.-Inner-Join)
* [3. Outer Join](#3.-Outer-Join)
* [4. Left Semi Join](#3.-Outer-Join)
* [5. Left Anti Join](#3.-Outer-Join)
* [6. Natural Join](#3.-Outer-Join)
* [7. Cross Join - Cartesian Join](#7.-Cross-Join---Cartesian-Join)
* [8. 조인시 유의할 점](#4.-조인시-유의할-점)
* [참고자료](#참고자료)

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

21/09/01 13:38:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/01 13:38:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## 1. 조인 유형

+ 스파크의 조인 타입
    + inner join, outer join, left outer join, right outer join
    + left semi join: 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우에는 키가 일치하는 왼쪽 데이터셋만 유지 
    + left anti join: 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 없는 경우에는 키가 일치하지 않는 왼쪽 데이터셋만 유지
    + natural join: 두 데이터셋에서 동일한 이름을 가진 컬럼을 암시적으로 결합하는 조인을 수행
    + cross join: 왼쪽 데이터셋의 모든 로우와 오른쪽 데이터 셋의 모든 로우를 조합

![join](images/join.png)

### JOIN 학습을 위해 상품은 단 하나만 구매할 수 있다고 가정하여 아래와 같은 테이블이 존재합니다
#### 정보 1. 고객은 4명이지만, 1명은 탈퇴하여 존재하지 않습니다
| 고객 아이디 (u_id) | 고객 이름 (u_name) | 고객 성별 (u_gender) |
| - | - | - |
| 1 | 정휘센 | 남 |
| 2 | 김싸이언 | 남 |
| 3 | 박트롬 | 여 |

#### 정보 2. 구매 상품은 3개이며, 탈퇴한 고객의 상품정보가 남아있습니다
| 구매 고객 아이디 (u_id) | 구매 상품 이름 (p_name) | 구매 상품 가격 (p_amount) |
| - | - | - |
| 2 | LG DIOS | 2,000,000 |
| 3 | LG Cyon | 1,800,000 |
| 4 | LG Computer | 4,500,000 |


In [2]:
user = spark.createDataFrame([
    (1, "정휘센", "남"),
    (2, "김싸이언", "남"),
    (3, "박트롬", "여")
]).toDF("u_id", "u_name", "u_gender")
user.printSchema()
display(user)
    
purchase = spark.createDataFrame([
    (2, "LG DIOS", 2000000),
    (3, "LG Cyon", 1800000),
    (4, "LG Computer", 4500000)
]).toDF("p_uid", "p_name", "p_amont")
purchase.printSchema()
display(purchase)

root
 |-- u_id: long (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)



u_id,u_name,u_gender
1,정휘센,남
2,김싸이언,남
3,박트롬,여


root
 |-- p_uid: long (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amont: long (nullable = true)



p_uid,p_name,p_amont
2,LG DIOS,2000000
3,LG Cyon,1800000
4,LG Computer,4500000


## 2. Inner Join

> join type 을 명시하지 않았을 때 기본적으로 내부조인을 수행함

* 왼쪽 테이블과 오른쪽 테이블에서 동일한 칼럼을 가져야함. 
* 참으로 평가되는 로우만 결합
* 세 번째 파라미터로 조인 타입을 명확하게 지정할 수 있음
* 두 테이블의 키가 중복되거나 여러 복사본으로 있다면 성능이 저하됨. 조인이 여러 키를 최소화하기 위해 일종의 카테시안 조인으로 변환되어 버림

### 2.1 구매 정보와 일치하는 고객 정보를 조인 (inner)

In [3]:
display(user.join(purchase, user.u_id == purchase.p_uid))
user.join(purchase, user.u_id == purchase.p_uid, "inner").count()

u_id,u_name,u_gender,p_uid,p_name,p_amont
3,박트롬,여,3,LG Cyon,1800000
2,김싸이언,남,2,LG DIOS,2000000


2

### <font color=green>1. [기본]</font> 고객 정보 f"{work_data}/tbl_user", 제품 정보 f"{work_data}/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 고객 테이블을 기준으로 어떤 제품을 구매하였는지 inner join 을 통해 조인해 주세요
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요

<details><summary>[실습1] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user.csv")
)
left.printSchema()
left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
right.printSchema()
right.show()

join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "inner")
answer.printSchema()
display(answer)
```

</details>


In [4]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

+----+----------+--------+--------+
|u_id|    u_name|u_gender|u_signup|
+----+----------+--------+--------+
|   1|    정휘센|      남|19700808|
|   2|  김싸이언|      남|19710201|
|   3|    박트롬|      여|19951030|
|   4|    청소기|      남|19770329|
|   5|유코드제로|      여|20021029|
|   6|  윤디오스|      남|20040101|
|   7|  임모바일|      남|20040807|
|   8|  조노트북|      여|20161201|
|   9|  최컴퓨터|      남|20201124|
+----+----------+--------+--------+

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

+----------+-----+----+-----------+--------+
|    p_time|p_uid|p_id|     p_name|p_amount|
+----------+-----+----+-----------+--------+
|1603651550|    0|1000|GoldStar TV|  100000|
|1603651550|    1|2000|    LG DIO

u_id,u_name,u_gender,u_signup,p_time,p_uid,p_id,p_name,p_amount
1,정휘센,남,19700808,1603651550,1,2000,LG DIOS,2000000
1,정휘센,남,19700808,1603694755,1,2001,LG Gram,1800000
2,김싸이언,남,19710201,1603673500,2,2002,LG Cyon,1400000
3,박트롬,여,19951030,1603652155,3,2003,LG TV,1000000
4,청소기,남,19770329,1603674500,4,2004,LG Computer,4500000
5,유코드제로,여,20021029,1603665955,5,2001,LG Gram,3500000
5,유코드제로,여,20021029,1603666155,5,2003,LG TV,2500000


## 3. Outer Join

> 왼쪽과 오른쪽의 모든 로우를 제공. 

* 왼쪽이나 오른쪽 DataFrame에 일치하는 로우가 없다면 해당 위치에 null을 삽입
* 공통 로우가 거의 없는 테이블에서 사용하면 결과값이 매우 커지고 성능 저하

### 3.1 모든 고객의 정보 구매 정보를 조인 (left_outer)

In [5]:
user.join(purchase, user.u_id == purchase.p_uid, "left_outer").orderBy(purchase.p_uid.asc())

u_id,u_name,u_gender,p_uid,p_name,p_amont
1,정휘센,남,null,null,null
2,김싸이언,남,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000


### 3-2. 모든 상품에 대한 고객 정보를 조인 (right_outer)

In [6]:
user.join(purchase, user.u_id == purchase.p_uid, "right_outer").orderBy(purchase.p_uid.asc())

u_id,u_name,u_gender,p_uid,p_name,p_amont
2,김싸이언,남,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000
null,null,null,4,LG Computer,4500000


### 3-3. 모든 고객과 상품에 대한 정보를 조인 (full_outer)

In [7]:
user.join(purchase, user.u_id == purchase.p_uid, "full_outer").orderBy(purchase.p_uid.asc())

u_id,u_name,u_gender,p_uid,p_name,p_amont
1,정휘센,남,null,null,null
2,김싸이언,남,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000
null,null,null,4,LG Computer,4500000


## 4. Left Semi Join

> 오른쪽에 존재하는 것을 기반으로 왼쪽 로우만 제공

* 값이 존재하는 지 확인 용도, 값이 있다면 왼쪽 DataFrame에 중복 키가 있더라도 해당 로우는 결과에 포함
* 기존 조인 기능과는 달리 DataFrame의 필터 기능과 유사
* 하나의 테이블만 확실히 고려되고, 다른 테이블은 조인 조건만 확인하기 때문에 성능이 매우 좋음


In [8]:
user.join(purchase, user.u_id == purchase.p_uid, "left_semi").orderBy(user.u_id.asc())

u_id,u_name,u_gender
2,김싸이언,남
3,박트롬,여


## 5. Left Anti Join
+ 왼쪽 세미 조인의 반대 개념, 즉 오른쪽 DataFrame의 어떤 값도 포함하지 않음
+ SQL의 NOT IN과 같은 스타일의 필터

In [9]:
user.join(purchase, user.u_id == purchase.p_uid, "left_anti").orderBy(user.u_id.asc())

u_id,u_name,u_gender
1,정휘센,남


## 6. Natural Join
> 조인하려는 컬럼을 암시적으로 추정

* 암시적인 처리는 언제나 위험하므로 비추천
* Python join 함수는 이 기능을 지원하지 않음


In [10]:
user.createOrReplaceTempView("user")
purchase.createOrReplaceTempView("purchase")
spark.sql("show tables")

database,tableName,isTemporary
,purchase,true
,user,true


In [11]:
user.printSchema()
purchase.printSchema()

root
 |-- u_id: long (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)

root
 |-- p_uid: long (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amont: long (nullable = true)



In [12]:
# 지정된 필드의 값이 일치하는 경우 해당 필드를 기준으로 조인
spark.sql("SELECT * FROM user NATURAL JOIN purchase")

u_id,u_name,u_gender,p_uid,p_name,p_amont
1,정휘센,남,2,LG DIOS,2000000
1,정휘센,남,3,LG Cyon,1800000
1,정휘센,남,4,LG Computer,4500000
2,김싸이언,남,2,LG DIOS,2000000
2,김싸이언,남,3,LG Cyon,1800000
2,김싸이언,남,4,LG Computer,4500000
3,박트롬,여,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000
3,박트롬,여,4,LG Computer,4500000


## 7. Cross Join - Cartesian Join

> 교차 조인은 조건절을 기술하지 않은 내부 조인을 의미
* 왼쪽의 모든 로우를 오른쪽의 모든 로우와 결합함(결과의 로우 수 = 왼쪽 로우 수 * 오른쪽 로우 수)
* 큰 데이터에서 사용할 경우 out-of-memory exception 발생. 가장 좋지 않은 성능을 가진 조인. 주의해서 사용하며 특정 사례에서만 사용해야함.

In [13]:
""" 크로스 조인이지만 조건을 설정해야 하며, 조건에 부합된 결과를 출력하여 inner조인과 동일."""
joinExpr = user.u_id == purchase.p_uid
joinType = "cross"
user.join(purchase, on=joinExpr, how=joinType)

u_id,u_name,u_gender,p_uid,p_name,p_amont
3,박트롬,여,3,LG Cyon,1800000
2,김싸이언,남,2,LG DIOS,2000000


In [14]:
user.crossJoin(purchase)

u_id,u_name,u_gender,p_uid,p_name,p_amont
1,정휘센,남,2,LG DIOS,2000000
1,정휘센,남,3,LG Cyon,1800000
1,정휘센,남,4,LG Computer,4500000
2,김싸이언,남,2,LG DIOS,2000000
2,김싸이언,남,3,LG Cyon,1800000
2,김싸이언,남,4,LG Computer,4500000
3,박트롬,여,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000
3,박트롬,여,4,LG Computer,4500000


### <font color=green>2. [기본]</font> 고객 정보 f"{work_data}/tbl_user", 제품 정보 f"{work_data}/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 모든 상품을 기준으로 구매하 고객정보를 조인해 주세요 (left: purchase, right: user, join: left_outer)
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
#### 6. 출력시에 상품 가격의 내림차순으로 정렬해 주세요

<details><summary>[실습2] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user.csv")
)
right.printSchema()
# right.show()

join_codition = left.p_uid == right.u_id
answer = left.join(right, join_codition, "left_outer")
answer.printSchema()
display(answer.orderBy(desc("p_amount")))
```

</details>


In [15]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)



p_time,p_uid,p_id,p_name,p_amount,u_id,u_name,u_gender,u_signup
1603674500,4,2004,LG Computer,4500000,4,청소기,남,19770329
1603665955,5,2001,LG Gram,3500000,5,유코드제로,여,20021029
1603666155,5,2003,LG TV,2500000,5,유코드제로,여,20021029
1603651550,1,2000,LG DIOS,2000000,1,정휘센,남,19700808
1603694755,1,2001,LG Gram,1800000,1,정휘센,남,19700808
1603673500,2,2002,LG Cyon,1400000,2,김싸이언,남,19710201
1603652155,3,2003,LG TV,1000000,3,박트롬,여,19951030
1603651550,0,1000,GoldStar TV,100000,null,null,null,null


### <font color=blue>3. [중급]</font> 고객 정보 f"{work_data}/tbl_user", 제품 정보 f"{work_data}/tbl_purchase" CSV 파일을 읽고
#### 1. 모든 고객을 기준으로 구매한 상품 정보를 조인해 주세요 (left: user, right: purchase, join: left_outer)
* 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
* 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
* 출력시에 상품 가격(tbl_purchase.p_amount)의 내림차순으로 정렬해 주세요
* 상품가격이 없는 경우에는 등록일자(tbl_user.u_signup)가 최신으로 정렬해 주세요
* 가능한 Structured API 를 사용하여 작성하되 최대한 간결하게 작성해 보세요

<details><summary>[실습3] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = spark.read.csv(f"{work_data}/tbl_user.csv", inferSchema=True, header=True)
right = spark.read.csv(f"{work_data}/tbl_purchase.csv", inferSchema=True, header=True)
answer = left.join(right, left.u_id == right.p_uid, "left_outer")
answer.printSchema()
display(answer.orderBy(desc("p_amount"), desc("u_signup")))
```

</details>


In [3]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



u_id,u_name,u_gender,u_signup,p_time,p_uid,p_id,p_name,p_amount
4,청소기,남,19770329,1603674500,4,2004,LG Computer,4500000
5,유코드제로,여,20021029,1603665955,5,2001,LG Gram,3500000
5,유코드제로,여,20021029,1603666155,5,2003,LG TV,2500000
1,정휘센,남,19700808,1603651550,1,2000,LG DIOS,2000000
1,정휘센,남,19700808,1603694755,1,2001,LG Gram,1800000
2,김싸이언,남,19710201,1603673500,2,2002,LG Cyon,1400000
3,박트롬,여,19951030,1603652155,3,2003,LG TV,1000000
9,최컴퓨터,남,20201124,null,null,null,null,null
8,조노트북,여,20161201,null,null,null,null,null
7,임모바일,남,20040807,null,null,null,null,null


### <font color=blue>4. [중급]</font> 고객 정보 f"{work_data}/tbl_user", 제품 정보 f"{work_data}/tbl_purchase" CSV 파일을 읽고
#### 1. 모든 고객과 모든 상품 정보가 출력될 수 있도록 조인해 주세요
* 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
* 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
* 출력시에 상품 가격(tbl_purchase.p_amount)의 내림차순으로 정렬해 주세요 (단, null 이 먼저 출력 되도록)
* 상품가격이 없는 경우에는 등록일자(tbl_user.u_signup)가 최신으로 정렬해 주세요 (단, null 이 마지막에 출력 되도록)
* 가능한 Structured API 를 사용하여 작성하되 최대한 간결하게 작성해 주세요

<details><summary>[실습4] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = spark.read.csv(f"{work_data}/tbl_user.csv", inferSchema=True, header=True)
right = spark.read.csv(f"{work_data}/tbl_purchase.csv", inferSchema=True, header=True)
answer = left.join(right, left.u_id == right.p_uid, "full_outer")
answer.printSchema()
display(answer.orderBy(desc_nulls_first("p_amount"), desc_nulls_last("u_signup")))
```

</details>


In [8]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



u_id,u_name,u_gender,u_signup,p_time,p_uid,p_id,p_name,p_amount
9,최컴퓨터,남,20201124,null,null,null,null,null
8,조노트북,여,20161201,null,null,null,null,null
7,임모바일,남,20040807,null,null,null,null,null
6,윤디오스,남,20040101,null,null,null,null,null
4,청소기,남,19770329,1603674500,4,2004,LG Computer,4500000
5,유코드제로,여,20021029,1603665955,5,2001,LG Gram,3500000
5,유코드제로,여,20021029,1603666155,5,2003,LG TV,2500000
1,정휘센,남,19700808,1603651550,1,2000,LG DIOS,2000000
1,정휘센,남,19700808,1603694755,1,2001,LG Gram,1800000
2,김싸이언,남,19710201,1603673500,2,2002,LG Cyon,1400000


### <font color=green>5. [기본]</font> 아래의 조인 연산 결과에서 null 값에 대한 치환을 해주세요
#### 1. 고객아이디(u_id)와 고객이름(u_name), 성별(u_gender), 가입일자(u_signup) 기본값을 채워주세요
##### u_id = 0, u_name = '미확인', u_gender = '미확인', u_signup = '19700101'

<details><summary>[실습5] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user.csv")
)
right.printSchema()
# right.show()

join_codition = left.p_uid == right.u_id
user_fill = { "u_id":0, "u_name":"미확인", "u_gender":"미확인", "u_signup":"19700101" }
answer = left.join(right, join_codition, "left_outer").na.fill(user_fill)
answer.printSchema()
display(answer.orderBy(asc("u_signup")))
```

</details>


In [19]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)
 |-- u_id: integer (nullable = false)
 |-- u_name: string (nullable = false)
 |-- u_gender: string (nullable = false)
 |-- u_signup: integer (nullable = true)



p_time,p_uid,p_id,p_name,p_amount,u_id,u_name,u_gender,u_signup
1603651550,0,1000,GoldStar TV,100000,0,미확인,미확인,19700101
1603651550,1,2000,LG DIOS,2000000,1,정휘센,남,19700808
1603694755,1,2001,LG Gram,1800000,1,정휘센,남,19700808
1603673500,2,2002,LG Cyon,1400000,2,김싸이언,남,19710201
1603674500,4,2004,LG Computer,4500000,4,청소기,남,19770329
1603652155,3,2003,LG TV,1000000,3,박트롬,여,19951030
1603665955,5,2001,LG Gram,3500000,5,유코드제로,여,20021029
1603666155,5,2003,LG TV,2500000,5,유코드제로,여,20021029


## 8. 조인시 유의할 점

In [20]:
u = spark.createDataFrame([
    (1, "정휘센", "남"),
    (2, "김싸이언", "남"),
    (3, "박트롬", "여")
]).toDF("id", "name", "gender")
u.printSchema()
display(u)
    
p = spark.createDataFrame([
    (2, "LG DIOS", 2000000),
    (3, "LG Cyon", 1800000),
    (4, "LG Computer", 4500000)
]).toDF("id", "name", "amount")
p.printSchema()
display(p)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)



id,name,gender
1,정휘센,남
2,김싸이언,남
3,박트롬,여


root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: long (nullable = true)



id,name,amount
2,LG DIOS,2000000
3,LG Cyon,1800000
4,LG Computer,4500000


### 8.1 중복 컬럼명 처리가 되지 않은 경우
> #### AnalysisException: "Reference 'id' is ambiguous, could be: id, id.;"

In [21]:
up = u.join(p, u.id == p.id)
up.show()
# up.select("id")

+---+--------+------+---+-------+-------+
| id|    name|gender| id|   name| amount|
+---+--------+------+---+-------+-------+
|  3|  박트롬|    여|  3|LG Cyon|1800000|
|  2|김싸이언|    남|  2|LG DIOS|2000000|
+---+--------+------+---+-------+-------+



### 3.2 중복 컬럼명 해결방안 - 데이터 프레임의 컬럼 명을 다르게 만든다

In [22]:
u1 = u.withColumnRenamed("id", "u_uid")
p1 = p.withColumnRenamed("id", "p_uid")
u1.printSchema()
p1.printSchema()

up = u1.join(p1, u1.u_uid == p1.p_uid)
display(up)
up.select("u_uid")

root
 |-- u_uid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- p_uid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: long (nullable = true)



u_uid,name,gender,p_uid,name,amount
3,박트롬,여,3,LG Cyon,1800000
2,김싸이언,남,2,LG DIOS,2000000


u_uid
3
2


### 3.3 중복 컬럼명 해결방안 - 조인 직후 중복 컬럼을 제거합니다

In [23]:
up = u.join(p, u.id == p.id).drop(p.id)
display(up)
display(up.select("id"))

id,name,gender,name,amount
3,박트롬,여,LG Cyon,1800000
2,김싸이언,남,LG DIOS,2000000


id
3
2


### <font color=red>6. [고급]</font> 고객 정보 f"{work_data}/tbl_user_id", 제품 정보 f"{work_data}/tbl_purchase_id" CSV 파일을 읽고
#### 1. 가장 비싼 `제품`을 구매한 고객의 고객정보와 제품정보를 출력해 주세요
* **고객(tbl_user) 테이블의 아이디도 id 이고 제품(tbl_purchase) 테이블의 아이디도 id** 인 점을 주의 하세요
* 최종 출력되는 컬럼은 고객아이디(u_id), 고객이름(u_name), 상품이름(p_name), 상품가격(p_amount) 이렇게 4개 컬럼입니다
* 상품가격 (p_amount) 내림차순으로 정렬해 주세요
* 가능한 Structured API 를 사용하여 작성하되 최대한 간결하게 작성해 보세요

<details><summary>[실습6] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = spark.read.csv(f"{work_data}/tbl_purchase_id.csv", inferSchema=True, header=True)
right = spark.read.csv(f"{work_data}/tbl_user_id.csv", inferSchema=True, header=True)
u_left = left.withColumnRenamed("id", "u_id")
u_right = right.withColumnRenamed("id", "p_uid")
answer = u_left.join(u_right, u_left.u_id == u_right.p_uid, "inner").where("p_amount > 0").select("u_id", "u_name", "p_name", "p_amount")
answer.printSchema()
display(answer.orderBy(desc("p_amount")))
```

</details>


In [11]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



u_id,u_name,p_name,p_amount
4,청소기,LG Computer,4500000
5,유코드제로,LG Gram,3500000
5,유코드제로,LG TV,2500000
1,정휘센,LG DIOS,2000000
1,정휘센,LG Gram,1800000
2,김싸이언,LG Cyon,1400000
3,박트롬,LG TV,1000000


## 9. 스파크의 조인 수행 방식

### 9.1 Broadcast Hash Join : **SmallSize** join **AnySize**
- 큰 데이터셋과 이보다 작은 데이터 셋 간의 조인은 큰 데이터셋의 파티션이 있는 모든 익스큐터에 작은 데이터셋이 브로드캐스트되어 수행
- 기본값: spark.sql.autoBroadcastJoinThreshold = 10mb 
  - spark.sql.autoBroadcastJoinThreshold 는 조인을 수행 할 때 모든 작업자 노드에 브로드캐스트되는 테이블의 최대 크기를 구성
  - 참고
    - [Spark Doc - performance Tuning](https://spark.apache.org/docs/latest/sql-performance-tuning.html)
    - [Does spark.sql.autoBroadcastJoinThreshold work for joins using Dataset's join operator?](https://stackoverflow.com/questions/43984068/does-spark-sql-autobroadcastjointhreshold-work-for-joins-using-datasets-join-op)
- 수행
  - 작은 테이블은 드라이버에 모였다가 다시 모든 노드에 복사
  - 작은 테이블은 메모리에 올라가고
  - 큰 테이블은 스트림을 통해서 조인 수행
  - 브로드캐스트 힌트 `person.join(broadcast(grudateProgram),Seq(“id”))`
    - `autoBroadcastJoinThreshold`에 관계없이 힌트가 있는 조인 측이 브로드캐스트됨. 
    - 조인의 양쪽에 브로드캐스트 힌트가 있으면 실제 크기가 더 작은 쪽이 브로드 캐스트됨. 
    - 힌트가 없고 테이블의 실제 물리적 추정값이 autoBroadcastJoinThreshold 보다 작으면, 해당 테이블은 모든 실행기 노드로 브로드캐스트됨.
- 성능 
  - 한 쪽의 데이터가 하나의 machine 에 fit-in-meory 될 정도로 작으면 성능 좋음.
    - table broadcast는 네트워크를 많이 사용하므로, 브로드캐스트 된 테이블이 크면 때때로 out-of-memory나 성능저하가 발생할 수 있음
  - 셔플링이 없기 때문에, 브로드캐스트 되는 쪽이 데이터가 작으면 다른 알고리즘보다 빠름. 
- 브로드캐스트 지원
  - Full outer join 은 지원하지 않음. 
  - letf-outer join 에서는 오른쪽 테이블만 브로드캐스트, right-outer join 에서는 왼쪽 테이블만 브로드캐스트 가능.

### 9.2 Shuffle hash join : **MiddleSize** join **LargeSize**
- 두 테이블 모두 shuffle 을 통해 노드에 분산되고
- 비교적 작은 테이블이 메모리 버퍼에 올라가고
- 큰 테이블은 스트림을 통해서 조인 수행
- 파티션이 전체 익스큐터로 분배
- 셔플은 비용이 많이 듦. 파티션과 셔플 배포가 최적으로 수행되는지 확인하기 위해 로직을 분석하는 것이 중요.
  - 큰 데이터는 join에 필요한 부분만 filtering 을 하거나
  - repartioning 을 고려해야함

- Map Reduce Fundamentals 유사
  - Map - 두 개의 서로 다른 Data frames/table
    - Output key를 join 조건에서 필드로 사용
    - Shuflle - output key로 두 데이터 세트를 섞음
  - Reduce - join 결과

![shuffle hash join](https://i.pinimg.com/originals/48/41/81/4841810dd7ad50397d566b8c9beb7875.jpg)

#### 성능을 최적화하려면
- join할 키가 균등하게 dirstribute되어있거나, 
- parallelism위한 적절한 수의 키가 있을 때
  
#### 성능이 나쁜 경우 - 고르지 않은 sharding 및 제한된 parallelism
- data skewness 처럼 하나의 단일 파티션이 다른 파티션에 비해 너무 많은 데이터를 가지고 있을 때
- 각 스테이트에서 50개 키만 셔플할 수 있음 -> 스파크 클러스터가 크면 고른 sharding과 parallelism 으로 해결 못함

![problem of shuffle hash join](https://image.slidesharecdn.com/optimizingsparksqljoins-170209164631/95/optimizing-apache-spark-sql-joins-11-638.jpg?cb=1486658917)

### 9.3 Sort merge join : **LargeSize** join **LargeSize**
- 일치하는 조인키를 sort할 수 있고, 브로드캐스트 조인, 셔플 해시 조인에 적합하지 않은 경우 사용
- shuffle hash join 과 비교했을 때, 클러스터에서 데이터 이동(shuffling)을 최소화함
- 수행
  - 두 테이블 모두 셔플 및 정렬이 발생하고
  - 그나마 작은 쪽이 버퍼를 하고 큰 쪽이 스트리밍으로 조인을 수행한다
  - partition 은 join 작업 전에 조인키 정렬
- 참고. [SortMergeJoinExec Binary Physical Operator for Sort Merge Join](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-SparkPlan-SortMergeJoinExec.html)

### 9.4 BroadcastNestedLoopJoin
- 적용 : 조인 키가 지정되어 있지 않고 브로드 캐스트 힌트가 있거나 조인의 한쪽이 브로드캐스트 될 수 있고, spark.sql.autoBroadcastJoinThreshold보다 작은 경우
- 브로드캐스트 된 데이터 세트가 크면 매우 느릴 수 있으며 OutOfMemoryExceptions을 일으킬 수 있음

---

### 9.5 테이블 크기에 따른 조인 동작방식

#### 큰 테이블과 큰 테이블 조인
+ 전체 노드 간 통신이 발생하는 셔플 조인이 발생됨

#### 큰 테이블과 작은 테이블 조인
+ 작은 DataFrame을 클러스터 전체 워커에 복제한 후 통신없이 진행
+ 모든 단일 노드에서 개별적으로 조인이 수행되므로 CPU가 가장 큰 병목 구간이 됨
+ broadcast 함수(힌트)를 통해 브로드캐스트 조인을 설정할 수 있으나 강제할 수는 없음(옵티마이저가 무시 가능)

In [25]:
from pyspark.sql.functions import broadcast
user.join(broadcast(purchase), user.u_id == purchase.p_uid).select("*")

u_id,u_name,u_gender,p_uid,p_name,p_amont
2,김싸이언,남,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000


In [26]:
user.join(broadcast(purchase), user.u_id == purchase.p_uid).select("*").explain()

== Physical Plan ==
*(2) BroadcastHashJoin [u_id#6L], [p_uid#44L], Inner, BuildRight, false
:- *(2) Project [_1#0L AS u_id#6L, _2#1 AS u_name#7, _3#2 AS u_gender#8]
:  +- *(2) Filter isnotnull(_1#0L)
:     +- *(2) Scan ExistingRDD[_1#0L,_2#1,_3#2]
+- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, true]),false), [id=#2486]
   +- *(1) Project [_1#38L AS p_uid#44L, _2#39 AS p_name#45, _3#40L AS p_amont#46L]
      +- *(1) Filter isnotnull(_1#38L)
         +- *(1) Scan ExistingRDD[_1#38L,_2#39,_3#40L]




#### 작은 테이블과 작은 테이블 조인
+ 스파크가 결정하도록 내버려두는 것이 제일 좋은 선택

### 9.6 고찰
- 브로드캐스트조인을 가능한 한 사용하고 조인 전에 관련없는 행을 조인 키로 필터링하여 불필요한 데이터 셔플링을 피하자
  - 필요한 경우 spark.sql.autoBroadcastJoinThreshold를 적절히 조정
- sort-merge join 이 default이고, 대부분의 시나리오에서 잘 수행됨. 
  - Shuffle Hash 조인이 Sort-Merge 조인보다 낫다고 확신이 있으면,Sort-Merge join을 비활성화해서 shuffle hash join이 수행되도록 함. 
    - builde size 가 stream size보다 작으면 Shuffle Hash 조인이 나음
- unique한 조인키가 없거나 조인키가 없는 조인은 수행비용이 비싸므로 최대한 피해야함

---

## 10. 설명 이해 위한 배경 지식

### 10.1 파티셔닝 partioning 
- 어떤 데이터를 어디에 저장할 것인지 제어할 수 있는 기능
- RDD는 데이터 파티션으로 구성되고 모든 연산은 RDD의 데이터 파티션에서 수행됨. 
- 파티션 개수는 RDD 트랜스포메이션 실행할 태스크 수에 직접적인 영향을 줌
  - 파티션 개수 너무 적으면 -> 많은 데이터에서 아주 일부의 CPU/코어만 사용 -> 성능 저하, 클러스터 제대로 활용 못함
  - 파티션 개수 너무 많으면 -> 실제 필요한 것보다 많은 자원을 사용 -> 멀티테넌트 환경에서는 자원 부족 현상 발생
- Partioner 에 의해 RDD 파티셔닝이 실행된. 파티셔너는 파티션 인덱스를 RDD 엘리먼트에 할당. 

### 10.2 Shuffling
- 파티셔너가 어떤 파티션을 사용하든 많은 연산이 RDD의 파티션 전체에 걸쳐 데이터 리파티셔닝Repartioning 이 발생함
  - 새로운 파티션이 생성되거나 파티션이 축소, 병합될 수 있음. 
- 리파티셔닝에 필요한 모든 데이터 이동을 **셔플링 Shuffling**이라고 함. 
  - **Shuffling 을 할 때, Disk I/O + Network I/O 과도하게 발생**
  - 셔플링은 계산을 동일 익스큐터의 메모리에서 더 이상 진행하지 않고 익스큐터 간에 데이터를 교환함 -> 많은 성능 지연을 초래할 수 있음 
  - 스파크 잡의 실행 프로세스를 결정, 잡이 스테이지로 분할되는 부분에 영향을 미침
  - 셔플링이 많을 수록 스파크 잡이 실행될 때 더 많은 스테이지가 발생하기 때문에 성능에 영향을 미침
- 리파티셔닝을 유발하는 연산은 **조인**, 리듀스, 그룹핑, 집계 연산이 있음

### 10.3 버켓팅 Bucketing
- Bucketing = pre-(shffle + sort) inputs on join Keys
- 각 파일에 저장된 데이터를 제어할 수 있는 또 다른 파일 조직화 기법
- 동일한 버킷 ID를 가진 데이터가 하나의 물리적 파티션에 모두 모여 있기 때문에 데이터를 읽을 때 셔플을 피할 수 있음
- **데이터가 이후 사용 방식에 맞춰 사전에 파티셔닝되므로 조인이나 집계할 때 발생하는 고비용의 셔플을 피할 수 있음.**
- 같은 키로 계속 조인이 발생하는 경우, 일별 누적으로 쌓여가는 테이블 을 버킷팅을 하면 효과를 볼 수 있음

### 10.4 Broadcast variable
- 브로드캐스트 변수는 모든 익스큐터에서 사용할 수 있는 공유 변수 shared variable
- 드라이버에서 한 번 생성되면 익스큐터에서만 읽을 수 있음. 
- 전체 데이터셋이 스파크 클러스터에서 전파될 수 있어서 익스큐터에서는 브로드캐스트 변수의 데이터에 접근할 수 있음
- **익스큐터 내부에서 실행되는 모든 태스크는 모두 브로드캐스트 변수에 접근할 수 있음**

---

## 참고자료

* [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
* [PySpark SQL Modules Documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)
* <a href="https://spark.apache.org/docs/3.0.1/api/sql/" target="_blank">PySpark 3.0.1 Builtin Functions</a>
* [PySpark Search](https://spark.apache.org/docs/latest/api/python/search.html)
* [Pyspark Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?#module-pyspark.sql.functions)
- [Spark SQL, DataFrames and Datasets Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
- [Spark summit 2017 - Hive Bucketing in Apache Spark with Tejas Patil](https://youtu.be/6BD-Vv-ViBw?t=30) / [slide](https://www.slideshare.net/databricks/hive-bucketing-in-apache-spark-with-tejas-patil) / [한글 요약본](https://www.notion.so/Hive-Bucketing-in-Apache-Spark-Tejas-Patil-9374879e0ca744cc8e7047e82cf5fdfa)
- [Spark summit 2017 - Optimizing Apache Spark SQL Joins: Spark Summit East talk by Vida Ha](https://www.youtube.com/watch?v=fp53QhSfQcI) / [slide](https://www.slideshare.net/databricks/optimizing-apache-spark-sql-joins)
- [Everyday I'm Shuffling - Tips for Writing Better Apache Spark Programs](https://www.youtube.com/watch?v=Wg2boMqLjCg)
- [Spark Memory Management by 0x0fff](https://0x0fff.com/spark-memory-management/)
- [Apache Spark에서 컬럼 기반 저장 포맷 Parquet(파케이) 제대로 활용하기](http://engineering.vcnc.co.kr/2018/05/parquet-and-spark/)
- [Understanding Database Sharding](https://www.digitalocean.com/community/tutorials/understanding-database-sharding)